### Metaclass

A metaclass is a class of a class. 
It allows you to customize class creation and behavior

What are the uses of metaclasses?
- automatically register a class
- validating class attributes and methods
- modify the class even before it is created
- enforce coding standards

### Example

In [ ]:
class MyMeta(type):

    def __new__(cls, name, bases, dct):
        dct['class_id'] = name.upper()
        print("Creating class: ")
        print(cls)
        print(name)
        print(bases)
        print(dct)
        return super().__new__(cls, name, bases, dct)

In [ ]:
class MyClass(metaclass=MyMeta):

    def greet(self):
        return f"{self.class_id}"

In [ ]:
m = MyClass()

In [ ]:
m.greet()

In [ ]:
dir(m)

##### What is __new__()?
- Special method -static method
- Creates a new instance of a class
- Gets called before the __init__
- Hence it can be used to customize the class creation itself
- Usually used when you want to control how classes are created
- Sometime you may want to inject some attributes, methods and validation logic in to the class

In [ ]:
class MyClass:

    def __new__(cls):
        print("Creating the class...")
        instance = super().__new__(cls)
        return instance

    def __init__(self):
        print("Initialization starts...")

In [ ]:
m = MyClass()

### Use Case

Let's say we want to make sure that the classes implement the required_attrs and it should be in the form of a list. In this case, we can create a metaclass that makes sure of this

In [ ]:
class ValidateAttrs(type):
    
    def __new__(cls, name, bases, dct):
        if 'required_attrs' not in dct:
            raise TypeError(f"Class {name} should implement required_attrs")
        if not isinstance(dct['required_attrs'], list):
            raise TypeError(f"required_attrs in Class {name} should be of type list")
        return super().__new__(cls, name, bases, dct)

In [ ]:
# Implementing without required_attrs: it will not even allow your to create a class
class A(metaclass=ValidateAttrs):

    def __init__(self):
        print("Class created")

In [ ]:
# With required_attrs
class A(metaclass=ValidateAttrs):

    required_attrs = ["name", "email"]

    def __init__(self):
        print("Class created")

In [ ]:
class Person(metaclass=ValidateAttrs):

    required_attrs = ['name', 'email']

    def __init__(self, **kargs):

        missing = [attr for attr in Person.required_attrs if attr not in kargs]
        if missing:
            raise ValueError(f"Missing attributes: {missing}")

        for key, value in dict(kargs).items():
            setattr(self, key, value)

In [ ]:
p = Person(name='Anil')

In [ ]:
p = Person(name="Anil", email="anil@ust.com")

##### The locals() Functions

In [ ]:
class Person(metaclass=ValidateAttrs):

    required_attrs = ['name', 'email']

    def __init__(self, **kargs):

        print('LOCALS: ', locals())
        
        missing = [attr for attr in Person.required_attrs if attr not in kargs]
        if missing:
            raise ValueError(f"Missing attributes: {missing}")

        for key, value in dict(kargs).items():
            setattr(self, key, value)

In [ ]:
p = Person(name="Anil", email="anil@ust.com")

### Use Case

Create a metaclass called EnforceFieldTypes that ensures any class using it must define a dictionary called field_types. This dictionary maps attribute names to their expected types (e.g., {"name": str, "age": int}).

The metaclass should inject a custom __init__ method into the class that:

    Accepts keyword arguments for all fields in field_types.
    Validates that each provided argument matches the expected type.
    Raises a TypeError if a field is missing or if the value type is incorrect.

Use this metaclass to implement a Student class.

In [ ]:
# Meta Class
class EnforceFieldTypes(type):

    def __new__(cls, name, bases, dct):

        # Extract field_types
        field_types = dct.get('field_types') # dct['field_types']
         
        # Validate the field_types
        if not isinstance(field_types, dict):
            raise TypeError(f"Class '{name}' must define 'field_types' dictionary")

        # We will also enforce __init__ to be defined in the class
        if '__init__' not in dct:
            raise TypeError(f"Class '{name}' must define '__init__' function")

        return super().__new__(cls, name, bases, dct)

In [ ]:
class A(metaclass=EnforceFieldTypes):

    def print(self):
        print("This is a class")

In [ ]:
# Let's add the field_types
class A(metaclass=EnforceFieldTypes):

    field_types = {'name':str, 'age':int}

    def print(self):
        print("This is a class")

In [ ]:
# Let's add the __init__()
class A(metaclass=EnforceFieldTypes):

    field_types = {'name':str, 'age':int}

    def __init__(self):
        self.name = "Anil"

    def print(self):
        print("This is a class")

##### Injecting Functions through Metaclass

In [ ]:
# Meta Class with __init__ injection
class EnforceFieldTypes(type):

    def __new__(cls, name, bases, dct):

        # Extract field_types
        field_types = dct.get('field_types') # dct['field_types']
         
        # Validate the field_types
        if not isinstance(field_types, dict):
            raise TypeError(f"Class '{name}' must define 'field_types' dictionary")

        '''
        # We will also enforce __init__ to be defined in the class
        # Since you're injecting the code, there is not need to validate
        if '__init__' not in dct:
            raise TypeError(f"Class '{name}' must define '__init__' function")
        '''
        # code to inject
        def __init__(self, **kargs):
            for field, expected_type in self.field_types.items():
                if field not in kargs:
                    raise TypeError(f"Missing required field: {field}")
                if not isinstance(kargs[field], expected_type):
                    raise TypeError(f"{field} must of type {expected_type}")
                setattr(self, field, kargs[field])

        # inject the code
        dct['__init__'] = __init__

        return super().__new__(cls, name, bases, dct)

In [ ]:
class Student(metaclass=EnforceFieldTypes):

    field_types = {
        "name"  : str,
        "age"   : int,
        "gpa"   : float,
        "major" : str
    }

    def get_details(self):
        return self.name, self.age, self.gpa, self.major

    def is_honors(self):
        return self.gpa >= 3.5

    def update_gpa(self, new_gpa):
        if not isinstance(new_gpa, float):
            raise TypeError("GPA must be a float")
        elif not (0.0 <= new_gpa <= 4.0):
            raise ValueError("GPA must be between 0.0 and 4.0")
        self.gpa = new_gpa

In [ ]:
s = Student(name='Ram', age=21, gpa=4.0, major='AI')
s.get_details()